In [ ]:
# -*- coding: utf-8 -*-
def train_sequential_model(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, time_steps=30):
    """
    Train a Sequential LSTM model for time series prediction.
    """
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dropout, Dense
    from tensorflow.keras.callbacks import EarlyStopping
    import numpy as np
    
    print("Starting sequential model training...")
    
    try:
        print(f"Input shapes - X_train: {X_train_scaled.shape}, X_test: {X_test_scaled.shape}")
        
        # Check if we have enough data for the sequence length
        if len(X_train_scaled) <= time_steps or len(X_test_scaled) <= time_steps:
            print(f"Warning: Not enough data for time_steps={time_steps}. Reducing time_steps.")
            time_steps = min(len(X_train_scaled) // 2, len(X_test_scaled) // 2, 20)
            print(f"Adjusted time_steps to {time_steps}")
        
        print("Creating training sequences...")
        # Reshape data for LSTM [samples, time_steps, features]
        X_train_seq, y_train_seq = [], []
        for i in range(time_steps, len(X_train_scaled)):
            X_train_seq.append(X_train_scaled[i-time_steps:i])
            y_train_seq.append(y_train_scaled[i])
        
        X_train_seq = np.array(X_train_seq)
        y_train_seq = np.array(y_train_seq)
        print(f"Training sequences created. X_train_seq shape: {X_train_seq.shape}")
        
        print("Creating test sequences...")
        # Prepare test data in the same way
        X_test_seq, y_test_seq = [], []
        for i in range(time_steps, len(X_test_scaled)):
            X_test_seq.append(X_test_scaled[i-time_steps:i])
            y_test_seq.append(y_test_scaled[i])
        
        X_test_seq = np.array(X_test_seq)
        y_test_seq = np.array(y_test_seq)
        print(f"Test sequences created. X_test_seq shape: {X_test_seq.shape}")
        
        # Print shapes for debugging
        print(f"Sequence shapes - X_train: {X_train_seq.shape}, y_train: {y_train_seq.shape}")
        print(f"Sequence shapes - X_test: {X_test_seq.shape}, y_test: {y_test_seq.shape}")
        
        if X_train_seq.shape[0] == 0 or X_test_seq.shape[0] == 0:
            raise ValueError(f"Created empty sequences. Try reducing time_steps (currently {time_steps}).")
        
        # Get the number of features
        n_features = X_train_scaled.shape[1]
        print(f"Number of features: {n_features}")
        
        print("Building model...")
        # Build the model
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, n_features)))
        model.add(Dropout(0.2))
        model.add(LSTM(units=50, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(units=50, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(units=50))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        
        print("Compiling model...")
        # Use run_eagerly=True to get better error messages
        model.compile(optimizer='adam', loss='mean_squared_error', run_eagerly=True)
        
        # Early stopping to prevent overfitting
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            mode='min',
            restore_best_weights=True
        )
        
        print("Training model...")
        # Train the model
        history = model.fit(
            X_train_seq, y_train_seq,
            epochs=50,
            batch_size=32,
            validation_data=(X_test_seq, y_test_seq),
            callbacks=[early_stopping],
            verbose=1
        )
        
        print("Model training completed successfully!")
        
        best_params = {
            'units': 50,
            'dropout': 0.2,
            'time_steps': time_steps
        }
        
        print("Returning model and results...")
        return model, history, X_test_seq, y_test_seq, best_params
        
    except Exception as e:
        print(f"ERROR in train_sequential_model: {e}")
        import traceback
        traceback.print_exc()
        # Return None values to indicate an error
        return None, None, None, None, None